<h1><font color='blue'>Lending Club</font></h1>

In [21]:
# Import the python environment libraries
import sklearn
import numpy as np
import pandas as pd
#import seaborn as sb
#from sklearn.linear_model import SGDClassifier
#from sklearn import metrics
from sklearn.cross_validation import cross_val_score, KFold, train_test_split
from sklearn import neighbors
from sklearn.metrics import classification_report
from __future__ import print_function
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
import time
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import random
import collections
from sklearn.naive_bayes import BernoulliNB
%matplotlib inline

In [27]:
# Import the data 
data = pd.read_csv("data.csv", parse_dates = [1])

#Adding additional dates to the dataframe 
df_date_split = pd.DatetimeIndex(data['issue_d'])
data['issue_year'] = df_date_split.year
data['issue_month'] = df_date_split.month

# Setting the initial features
df_features = data.copy()
del df_features['id']
del df_features['new_int_rate']
del df_features['issue_d']



# Hash the dataset until we determine columns that require it
data['grade'] = data['grade'].apply(hash)
data['sub_grade'] = data['sub_grade'].apply(hash)
data['home_ownership'] = data['home_ownership'].apply(hash)
data['verification_status'] = data['verification_status'].apply(hash)
data['loan_status'] = data['loan_status'].apply(hash)
data['pymnt_plan'] = data['pymnt_plan'].apply(hash)
data['purpose'] = data['purpose'].apply(hash)
data['addr_state'] = data['addr_state'].apply(hash)

# Preview the data
data.head(1)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies,new_int_rate,int_bucket,issue_year,issue_month
0,1077501,1296599,5000,5000,4975.0,36,0.1065,162.87,-141909181,406699638,...,0.0,0.0,42384.0,171.62,42659.0,0,11,2,2016,12


In [28]:
#use this function to randomly select the features
#we should look at a few thousand scenarios as we explore the best scores
def RandomFeatures():
    features = df_features.columns.values.tolist()
    results = list()
    for f in features:
        randomNumber = random.randint(1, 10)
        if randomNumber >= 5:
            results.append(f)
        
    return results


In [29]:
#automated classifier support
'''
train_c = features to use
model = KNN,LOG_REG, ETC
ts = test size (.4)
'''
def PredStats(train_c,model,ts):
    
    X = np.matrix(data[train_c])
    y = (np.array(data['new_int_rate'])).T

    #train the data
    X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=ts)

    if model == 'KNN_REG':
        #knn classifier
        knearest_clf = neighbors.KNeighborsRegressor(n_neighbors=2, weights='uniform') 
        knearest_clf = knearest_clf.fit(X_train, y_train)
        y_pred_class = knearest_clf.predict(X_test)
    elif model =='LOG_REG':
        #logistic regression classifier
        logreg = LogisticRegression()
        logreg = logreg.fit(X_train, y_train)
        y_pred_class = logreg.predict(X_test)
    elif model =='BAYES':
        #bayes classifier
        bayes_clf = BernoulliNB()
        #fit classifier
        bayes_clf.fit(X_train, y_train)
        y_pred_class = bayes_clf.predict(X_test)
    else:
        print('Input a correct model')
        #break

    if model == 'LOG_REG':
        print('Coefficients: \n',logreg.coef_)
        print('MSE:',metrics.mean_squared_error(y_test,y_pred_class))
        print('Variance score: ' , logreg.score(X,y))
    elif model == 'KNN_REG':
        print ('KNN Score: ',knearest_clf.score(X,y))
        print('MSE:',metrics.mean_squared_error(y_test,y_pred_class))
        print('Variance score: ' , knearest_clf.score(X,y))
    elif model == 'BAYES':
        scores = cross_val_score(bayes_clf, X, y, cv=2, scoring='accuracy')
        print('Scores: ',scores)
        print('Score mean:',scores.mean())
    print (train_c)

In [30]:
# Look for the best score
for i in range(5):
    features= RandomFeatures()
    PredStats(features,"BAYES",.4)

Scores:  [ 0.24510282  0.2663319 ]
Score mean: 0.255717359883
['member_id', 'loan_amnt', 'funded_amnt', 'term', 'installment', 'grade', 'sub_grade', 'emp_length', 'verification_status', 'loan_status', 'purpose', 'revol_util', 'out_prncp', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'pub_rec_bankruptcies', 'int_bucket', 'issue_year']
Scores:  [ 0.17668843  0.19642161]
Score mean: 0.186555019395
['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'grade', 'annual_inc', 'verification_status', 'purpose', 'zip_code', 'delinq_2yrs', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp_inv', 'total_rec_prncp', 'total_rec_int', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'int_bucket', 'issue_month']
Scores:  [ 0.22419842  0.22187012]
Score mean: 0.223034273006
['funded_amnt', 'funded_